# Energy system optimisation with oemof - how to collect and store results

### Import necessary modules

In [1]:
import os
import pandas as pd
from oemof.solph import (Sink, Source, Transformer, Bus, Flow, Model,
                         EnergySystem)
import oemof.outputlib as outputlib
import pickle

### Specify solver

In [2]:
solver = 'cbc'

### Create an energy system and optimize the dispatch at least costs.

In [3]:
# initialize and provide data
datetimeindex = pd.date_range('1/1/2016', periods=24*10, freq='H')
energysystem = EnergySystem(timeindex=datetimeindex)
filename = 'input_data.csv'
data = pd.read_csv(filename, sep=",")

### Create and add components to energysystem

In [4]:
# resource buses
bcoal = Bus(label='coal', balanced=False)
bgas = Bus(label='gas', balanced=False)
boil = Bus(label='oil', balanced=False)
blig = Bus(label='lignite', balanced=False)

# electricity and heat
bel = Bus(label='bel')
bth = Bus(label='bth')

energysystem.add(bcoal, bgas, boil, blig, bel, bth)

In [5]:
# an excess and a shortage variable can help to avoid infeasible problems
energysystem.add(Sink(label='excess_el', inputs={bel: Flow()}))
# shortage_el = Source(label='shortage_el',
#                      outputs={bel: Flow(variable_costs=200)})

# sources
energysystem.add(Source(label='wind', outputs={bel: Flow(
    actual_value=data['wind'], nominal_value=66.3, fixed=True)}))

energysystem.add(Source(label='pv', outputs={bel: Flow(
    actual_value=data['pv'], nominal_value=65.3, fixed=True)}))

# demands (electricity/heat)
energysystem.add(Sink(label='demand_el', inputs={bel: Flow(
    nominal_value=85, actual_value=data['demand_el'], fixed=True)}))

energysystem.add(Sink(label='demand_th',
                 inputs={bth: Flow(nominal_value=40,
                                   actual_value=data['demand_th'],
                                   fixed=True)}))

In [6]:
# power plants
energysystem.add(Transformer(
    label='pp_coal',
    inputs={bcoal: Flow()},
    outputs={bel: Flow(nominal_value=20.2, variable_costs=25)},
    conversion_factors={bel: 0.39}))

energysystem.add(Transformer(
    label='pp_lig',
    inputs={blig: Flow()},
    outputs={bel: Flow(nominal_value=11.8, variable_costs=19)},
    conversion_factors={bel: 0.41}))

energysystem.add(Transformer(
    label='pp_gas',
    inputs={bgas: Flow()},
    outputs={bel: Flow(nominal_value=41, variable_costs=40)},
    conversion_factors={bel: 0.50}))

energysystem.add(Transformer(
    label='pp_oil',
    inputs={boil: Flow()},
    outputs={bel: Flow(nominal_value=5, variable_costs=50)},
    conversion_factors={bel: 0.28}))

In [7]:
# combined heat and power plant (chp)


In [8]:
# heat pump with a coefficient of performance (COP) of 3
b_heat_source = Bus(label='b_heat_source')
energysystem.add(b_heat_source)

energysystem.add(Source(label='heat_source', outputs={b_heat_source: Flow()}))

cop = 3
energysystem.add(Transformer(
    label='heat_pump',
    inputs={bel: Flow(),
            b_heat_source: Flow()},
    outputs={bth: Flow(nominal_value=10)},
    conversion_factors={bel: 1/3, b_heat_source: (cop-1)/cop}))

### Optimization

In [9]:
# create optimization model based on energy_system
optimization_model = Model(energysystem=energysystem)

# solve problem
optimization_model.solve(solver=solver,
                         solve_kwargs={'tee': True, 'keepfiles': False})

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Aug 21 2017 

command line - /usr/bin/cbc -printingOptions all -import /home/local/RL-INSTITUT/okan.akca/Schreibtisch/oemof-examples/oemof_examples/oemof.solph/v0.3.x/jupyter_tutorials/tmprkmmoipf.pyomo.lp -stat=1 -solve -solu /home/local/RL-INSTITUT/okan.akca/Schreibtisch/oemof-examples/oemof_examples/oemof.solph/v0.3.x/jupyter_tutorials/tmprkmmoipf.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve determined that the problem was infeasible with tolerance of 1e-08
Presolved model looks infeasible - will use unpresolved


Problem has 2161 rows, 3121 columns (960 with objective) and 5041 elements
There are 1441 singletons with no objective 
Column breakdown:
1921 of type 0.0->inf, 1200 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 0 of type 0.0->1.0 
Row breakdown:
1680 of type E 0.0, 1 of type E 1.0,

ERROR:root:Optimization failed with status ok and terminal condition infeasible


{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 0, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'Termination condition': 'infeasible', 'Error rc': 0, 'Time': 0.0820760726928711}]}

### Write results into energysystem.results object for later

In [10]:
energysystem.results['main'] = outputlib.processing.results(optimization_model)
energysystem.results['meta'] = outputlib.processing.meta_results(optimization_model)

ERROR:pyomo.core:evaluating expression: No value for uninitialized NumericValue object flow[pp_coal,bel,0]
    (expression: 25*flow[pp_coal,bel,0] + 25*flow[pp_coal,bel,1] + 25*flow[pp_coal,bel,2] + 25*flow[pp_coal,bel,3] + 25*flow[pp_coal,bel,4] + 25*flow[pp_coal,bel,5] + 25*flow[pp_coal,bel,6] + 25*flow[pp_coal,bel,7] + 25*flow[pp_coal,bel,8] + 25*flow[pp_coal,bel,9] + 25*flow[pp_coal,bel,10] + 25*flow[pp_coal,bel,11] + 25*flow[pp_coal,bel,12] + 25*flow[pp_coal,bel,13] + 25*flow[pp_coal,bel,14] + 25*flow[pp_coal,bel,15] + 25*flow[pp_coal,bel,16] + 25*flow[pp_coal,bel,17] + 25*flow[pp_coal,bel,18] + 25*flow[pp_coal,bel,19] + 25*flow[pp_coal,bel,20] + 25*flow[pp_coal,bel,21] + 25*flow[pp_coal,bel,22] + 25*flow[pp_coal,bel,23] + 25*flow[pp_coal,bel,24] + 25*flow[pp_coal,bel,25] + 25*flow[pp_coal,bel,26] + 25*flow[pp_coal,bel,27] + 25*flow[pp_coal,bel,28] + 25*flow[pp_coal,bel,29] + 25*flow[pp_coal,bel,30] + 25*flow[pp_coal,bel,31] + 25*flow[pp_coal,bel,32] + 25*flow[pp_coal,bel,33] + 25

ERROR: evaluating expression: No value for uninitialized NumericValue object
    flow[pp_coal,bel,0]
        (expression: 25*flow[pp_coal,bel,0] + 25*flow[pp_coal,bel,1] +
        25*flow[pp_coal,bel,2] + 25*flow[pp_coal,bel,3] +
        25*flow[pp_coal,bel,4] + 25*flow[pp_coal,bel,5] +
        25*flow[pp_coal,bel,6] + 25*flow[pp_coal,bel,7] +
        25*flow[pp_coal,bel,8] + 25*flow[pp_coal,bel,9] +
        25*flow[pp_coal,bel,10] + 25*flow[pp_coal,bel,11] +
        25*flow[pp_coal,bel,12] + 25*flow[pp_coal,bel,13] +
        25*flow[pp_coal,bel,14] + 25*flow[pp_coal,bel,15] +
        25*flow[pp_coal,bel,16] + 25*flow[pp_coal,bel,17] +
        25*flow[pp_coal,bel,18] + 25*flow[pp_coal,bel,19] +
        25*flow[pp_coal,bel,20] + 25*flow[pp_coal,bel,21] +
        25*flow[pp_coal,bel,22] + 25*flow[pp_coal,bel,23] +
        25*flow[pp_coal,bel,24] + 25*flow[pp_coal,bel,25] +
        25*flow[pp_coal,bel,26] + 25*flow[pp_coal,bel,27] +
        25*flow[pp_coal,bel,28] + 25*flow[pp_coal,bel,29

ValueError: No value for uninitialized NumericValue object flow[pp_coal,bel,0]

In [ ]:
string_results = outputlib.views.convert_keys_to_strings(energysystem.results['main'])

### Save results - Dump the energysystem (to ~/home/user/.oemof by default)
Specify path and filename if you do not want to overwrite

In [ ]:
energysystem.dump(dpath=None, filename=None)